In [77]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [78]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2023-02-08 16:18:56--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.1’

book-crossings.zip. 100%[===================>]  24.88M   131MB/s    in 0.2s    

2023-02-08 16:18:56 (131 MB/s) - ‘book-crossings.zip.1’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [79]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

#Info

In [80]:
# Normalizing info
#So I will implement clustering, each dimension will be a user, and the value in that dimension will be the callification, if it's not known then could i make a dnn? I don't think so, it will need a dnn per book 
df_ratings.head(10)

#print(df_books.head(10))

#print('\n Number of Rates per user \n',df_ratings['user'].value_counts())
#print('\n Number of Rates per User statistics \n',df_ratings['user'].value_counts().describe())
#print('\n Memory usage \n',df_ratings.memory_usage())

#print('\nRates per user \n',df_ratings['user'].describe())
#print('\nRates per user \n',df_ratings[['user','rating']].describe())

#print('\nRates per user \n',df_ratings['user'].value_counts)
#print('\n Rate per User above 100 \n',df_ratings['user'].value_counts().describe())

#print('\n Books per author \n',df_books['author'].value_counts())
#print('\n Books per author statistics \n',df_books['author'].value_counts().describe())


,user,isbn,rating
0,276725,034545104X,0.0
1,276726,0155061224,5.0
2,276727,0446520802,0.0
3,276729,052165615X,3.0
4,276729,0521795028,6.0
5,276733,2080674722,0.0
6,276736,3257224281,8.0
7,276737,0600570967,6.0
8,276744,038550120X,7.0
9,276745,342310538,10.0


## Decisions

There are a few decisions taken here only to fullfil the test. It's supposed to be filtered by ISBN, so if a book have many editions it can be filtered, something common between classic books. 

I have tested filtering by Title with fewer reviews than 100 and I arrive to different results, with better distances. Also it has been decided to use cosine distance and not euler one

In [81]:
from pandas.core.window.expanding import Axis

def labelWithLessAppearanceThan(dataframe,fewAppearance,column, more=False):
  justColumn = dataframe.drop([col for col in dataframe.columns if col != column], axis='columns')  #.drop('isbn', axis='columns')
  justColumn["count"] = 0
  countGroupColumn = justColumn.groupby(column).count()
  query='count '
  if more:
    query+='>'
  else:
    query+='<'
  query+=str(fewAppearance)

  return countGroupColumn.query(query).index
 
#print(labelWithLessAppearanceThan(df_books,6,'title',more=True))


ratingExtended=df_books[["isbn","title"]].merge(df_ratings,on=['isbn'])

userWithLessR = labelWithLessAppearanceThan(df_ratings,200,'user')
dfFiltered = ratingExtended[~ratingExtended["user"].isin(userWithLessR)]


#Books are the same based if they have the same title

#idBook='title'
#bookWithLessR = labelWithLessAppearanceThan(ratingExtended,100,idBook) #It will verify condition before filtering

#I have tried this and it gives different results with better distances, it's unfair a book with many editions could be filtered, for example a classic

#------------------------------------------------- 


#Books are different if they have differents isbn

idBook='isbn'
bookWithLessR = labelWithLessAppearanceThan(ratingExtended,100,idBook) #It doesn't use the first filtered for selecting
#-------------------------------------------------------

dfFiltered = dfFiltered[~dfFiltered[idBook].isin(bookWithLessR)]
print(dfFiltered)







              isbn                                              title    user  \
98      0440234743                                      The Testament  277478   
103     0440234743                                      The Testament    2977   
105     0440234743                                      The Testament    3363   
108     0440234743                                      The Testament    7346   
112     0440234743                                      The Testament    9856   
...            ...                                                ...     ...   
505464  0515135739  Eleventh Hour: An FBI Thriller (FBI Thriller (...  236283   
505469  0515135739  Eleventh Hour: An FBI Thriller (FBI Thriller (...  251613   
505470  0515135739  Eleventh Hour: An FBI Thriller (FBI Thriller (...  252071   
505473  0515135739  Eleventh Hour: An FBI Thriller (FBI Thriller (...  256407   
505474  0515135739  Eleventh Hour: An FBI Thriller (FBI Thriller (...  262399   

        rating  
98        

## Filter Ratings with rate 0
It's writen that ratings are from 1 to 10

In [82]:
#It says callification its from 1 to 10
#dfFiltered['rating'].describe()
#This shows than more than the 50% callifies with 0, so it's a possible value or an error?
#belowZero= dfFiltered.query('rating <= 0')
#withoutZeros = dfFiltered.drop(belowZero.index)
#dfFiltered=withoutZeros#maybe not, too little information is left, if i do it at the beggining only 2166 ratings will be left, and if i do it at the end i will have 17391
#I wanted to do this so I can normalize rating for each user(not for everyone the difference between a 5 stars and a 7 stars are the same) for this i needed to delete false 0s
#I have to filter after transforming it to a new table, normalizing by column

#Generate Matrix

In [83]:
matrix = dfFiltered.pivot_table(index=['title'],columns=['user'],values='rating',
                                #fill_value=0,#if i want to normalize i should not assign zeros
                                )
print(matrix.head())


user                                                254     2276    2766    \
title                                                                        
1984                                                   9.0     NaN     NaN   
1st to Die: A Novel                                    NaN     NaN     NaN   
2nd Chance                                             NaN    10.0     NaN   
4 Blondes                                              NaN     NaN     NaN   
A Beautiful Mind: The Life of Mathematical Geni...     NaN     NaN     NaN   

user                                                2977    3363    4017    \
title                                                                        
1984                                                   NaN     NaN     NaN   
1st to Die: A Novel                                    NaN     NaN     NaN   
2nd Chance                                             NaN     NaN     NaN   
4 Blondes                                              NaN     

#Normalizing rattings

In [84]:
#matrix= (matrix-matrix.mean()) / matrix.std() #If i normalize i have other results
matrix=matrix.fillna(0.)
matrix.head()


user,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Beautiful Mind: The Life of Mathematical Genius and Nobel Laureate John Nash,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [85]:
model = NearestNeighbors(metric='cosine',
                         n_neighbors=5) #I use cosine because the values are between 0 and 1 in the test, with other methods it returns distances of 60's, I don't find much sense using cosine
model.fit(matrix)
print(matrix.shape)



(673, 888)


## Function recommended Titles

In [86]:
# function to return recommended books - this will be tested
def get_recommends(title = ""):
  try:
    book = matrix.loc[title]
  except KeyError as e:
    print('The given book', e, 'does not exist')
    return

  distance, indice = model.kneighbors([book.values], n_neighbors=6)

  recommended_books = pd.DataFrame({
      'title'   : matrix.iloc[indice[0]].index.values[1:],
      'distance': distance[0][1:]
    }).values 
  #It looks it recommend first the less similar book, i think that's weird
  return [title, recommended_books[::-1]]

print(get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))"))

["Where the Heart Is (Oprah's Book Club (Paperback))", array([["I'll Be Seeing You", 0.8016210794448853],
       ['The Weight of Water', 0.7708583474159241],
       ['The Surgeon', 0.7699410915374756],
       ['I Know This Much Is True', 0.7677075266838074],
       ['The Lovely Bones: A Novel', 0.7234864234924316]], dtype=object)]


In [87]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", array([["I'll Be Seeing You", 0.8016210794448853],
       ['The Weight of Water', 0.7708583474159241],
       ['The Surgeon', 0.7699410915374756],
       ['I Know This Much Is True', 0.7677075266838074],
       ['The Lovely Bones: A Novel', 0.7234864234924316]], dtype=object)]
You passed the challenge! 🎉🎉🎉🎉🎉
